In [1]:
%autoreload 2

In [45]:
from argparse import Namespace
from collections import defaultdict
import copy
from datetime import datetime
import difflib
import gzip
import itertools
import os
import pickle
import sys
import typing

from IPython.display import display, Markdown, HTML  # type: ignore
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import polars as pl
import tatsu
import tqdm.notebook as tqdmn


sys.path.append(os.path.abspath('..'))
sys.path.append(os.path.abspath('../reward-machine'))

import compile_predicate_statistics
import compile_predicate_statistics_split_args

In [41]:
cache_dir = compile_predicate_statistics.get_project_dir() + '/reward-machine/caches'

regular_df = pd.read_pickle(os.path.join(cache_dir, 'predicate_statistics.pkl'))
split_args_df = pd.read_pickle(os.path.join(cache_dir, 'predicate_statistics_split_args.pkl'))

stats = compile_predicate_statistics.CommonSensePredicateStatistics(cache_dir)
split_args_stats = compile_predicate_statistics_split_args.CommonSensePredicateStatisticsSplitArgs(cache_dir)

Loaded data with shape (43521, 6) from /Users/guydavidson/projects/game-generation-modeling/reward-machine/caches/predicate_statistics.pkl
Loaded data with shape (43521, 8) from /Users/guydavidson/projects/game-generation-modeling/reward-machine/caches/predicate_statistics_split_args.pkl


Debugging `(and (not (in_motion ?b)) (in ?h ?b)))`

In [26]:
trace_id = '7r4cgxJHzLJooFaMG1Rd-gameplay-attempt-1-rerecorded'
ball_id = 'Dodgeball|+00.19|+01.13|-02.80'
bin_id = 'GarbageCan|+00.75|-00.03|-02.74'
arg_ids = (bin_id, ball_id)
mapping = {'?b': ['ball'], '?h': ['hexagonal_bin']}
arg_types = ('garbagecan' ,'dodgeball')
regular_df[(regular_df.trace_id == trace_id) & (regular_df.predicate == 'in') & (regular_df.arg_ids == arg_ids)]

,predicate,arg_ids,arg_types,trace_id,domain,intervals
21566,in,"(GarbageCan|+00.75|-00.03|-02.74, Dodgeball|+0...","(garbagecan, dodgeball)",7r4cgxJHzLJooFaMG1Rd-gameplay-attempt-1-rereco...,many,"[[3304, 3640]]"


In [56]:
stats._union_intervals([[1, 2], [3, 4]], [])

[[1, 4]]

In [34]:
stats._invert_intervals(regular_df[(regular_df.trace_id == trace_id) & (regular_df.predicate == 'in_motion') & (regular_df.arg_ids == (ball_id,))].intervals.values[0],
                        stats.trace_lengths[trace_id])

[[0, 519],
 [559, 598],
 [624, 726],
 [730, 1190],
 [1191, 1623],
 [1679, 1884],
 [1924, 2081],
 [2090, 2312],
 [2367, 2474],
 [2568, 2946],
 [2985, 3054],
 [3118, 3145],
 [3176, 3200],
 [3256, 3289],
 [3311, 3312],
 [3327, 3537],
 [3538, 3640]]

In [36]:
split_args_df[(split_args_df.trace_id == trace_id) & (split_args_df.predicate == 'in') & (split_args_df.arg_1_id == bin_id) & (split_args_df.arg_2_id == ball_id)]

,predicate,arg_1_id,arg_1_type,arg_2_id,arg_2_type,trace_id,domain,intervals
21566,in,GarbageCan|+00.75|-00.03|-02.74,garbagecan,Dodgeball|+00.19|+01.13|-02.80,dodgeball,7r4cgxJHzLJooFaMG1Rd-gameplay-attempt-1-rereco...,many,"[[3304, 3640]]"


In [43]:
ball_not_in_motion_int = split_args_stats._invert_intervals(
    split_args_df[(split_args_df.trace_id == trace_id) & (split_args_df.predicate == 'in_motion') & (split_args_df.arg_1_id == ball_id)].intervals.values[0],
    split_args_stats.trace_lengths_and_domains[trace_id][0],
)

ball_not_in_motion_int

[[0, 519],
 [559, 598],
 [624, 726],
 [730, 1190],
 [1191, 1623],
 [1679, 1884],
 [1924, 2081],
 [2090, 2312],
 [2367, 2474],
 [2568, 2946],
 [2985, 3054],
 [3118, 3145],
 [3176, 3200],
 [3256, 3289],
 [3311, 3312],
 [3327, 3537],
 [3538, 3640]]

In [44]:
ball_in_bin_int = split_args_df[(split_args_df.trace_id == trace_id) & (split_args_df.predicate == 'in') & (split_args_df.arg_1_id == bin_id) & (split_args_df.arg_2_id == ball_id)].intervals.values[0]

split_args_stats._intersect_intervals(ball_in_bin_int, ball_not_in_motion_int)

[[3311, 3312], [3327, 3537], [3538, 3640]]

In [47]:
single_trace_split_args_df = split_args_df[split_args_df['trace_id'] == trace_id]

single_trace_split_args_stats = compile_predicate_statistics_split_args.CommonSensePredicateStatisticsSplitArgs(cache_dir)
single_trace_split_args_stats.data = pl.from_pandas(single_trace_split_args_df)
single_trace_split_args_stats.data.shape

Loaded data with shape (43521, 8) from /Users/guydavidson/projects/game-generation-modeling/reward-machine/caches/predicate_statistics_split_args.pkl


(901, 8)

In [49]:
DEFAULT_GRAMMAR_PATH = "../dsl/dsl.ebnf"
grammar = open(DEFAULT_GRAMMAR_PATH).read()
grammar_parser = typing.cast(tatsu.grammars.Grammar, tatsu.compile(grammar))

game = open(compile_predicate_statistics.get_project_dir() + '/reward-machine/games/ball_to_bin_from_bed.txt').read()
game_ast = grammar_parser.parse(game) 

test_pred_1 = game_ast[4][1]['preferences'][0]['definition']['forall_pref']['preferences']['pref_body']['body']['exists_args']['then_funcs'][1]['seq_func']['hold_pred']

# should be: (and (not (in_motion ?b)) (in ?h ?b)))
test_pred_2 = game_ast[4][1]['preferences'][0]['definition']['forall_pref']['preferences']['pref_body']['body']['exists_args']['then_funcs'][2]['seq_func']['once_pred']

In [55]:
test_mapping = {"?b": ["ball"], "?h": ["hexagonal_bin"]}
test_out = single_trace_split_args_stats.filter(test_pred_2, test_mapping)
test_out

{('7r4cgxJHzLJooFaMG1Rd-gameplay-attempt-1-rerecorded',
  ('?b->Dodgeball|+00.19|+01.13|-02.80',
   '?h->GarbageCan|+00.75|-00.03|-02.74')): [[3327, 3537], [3538, 3640]],
 ('7r4cgxJHzLJooFaMG1Rd-gameplay-attempt-1-rerecorded',
  ('?b->Dodgeball|+00.70|+01.11|-02.80',
   '?h->GarbageCan|+00.75|-00.03|-02.74')): [[2530, 3205],
  [3207, 3295],
  [3311, 3314],
  [3327, 3537],
  [3547, 3640]]}